- 파이썬이 데커레이터 구문을 평가하는 방식  
- 변수가 지역변수인지 파이썬이 판단하는 방식
- 클로저의 존재 이유와 작동 방식
- nonlocal로 해결할 수 있는 문제
  
  
- 잘 작동하는 데커레이터 구현하기
- 표준 라이브러리에서 제공하는 재미있는 데커레이터들
- 매개변수화된 데커레이터 구현하기

### 7.1 데커레이터 기본 지식
1. 데커레이터는 데커레이트된 하수를 다른 함수로 대체하는 능력이 있다.
2. 데커레이터는 모듈이 로딩될 때 바로 실행된다.

In [1]:
@decorate
def target():
    print ('running target()')
    
def target():
    print ('running target()')
target = decorate(target)

NameError: name 'decorate' is not defined

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print("running target()")

target()
target

running inner()


<function __main__.deco.<locals>.inner>

### 7.2 파이썬이 데커레이터를 실행하는 시점

In [9]:
registry = []

def register(func):
    print ('running register(%s)' %func)
    registry.append(func)
    return func

@register
def f1():
    print ("running f1()")

@register
def f2():
    print ("running f2()")

def f3():
    print ("running f3()")
    
def main():
    print ("running main()")
    print ("registry -> ", registry)
    f1()
    f2()
    f3()
main()
    

running register(<function f1 at 0x7fd5d4b42510>)
running register(<function f2 at 0x7fd5cc1556a8>)
running main()
registry ->  [<function f1 at 0x7fd5d4b42510>, <function f2 at 0x7fd5cc1556a8>]
running f1()
running f2()
running f3()


### 7.3 데커레이터로 개선한 전략 패턴

In [1]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order: #콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return  0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

In [2]:
# END STRATEGY
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10,1.5), LineItem('watermellon', 5, 5.0)]

print(Order(joe, cart, fildelity))
print(Order(ann, cart, fildelity))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print (Order(ann, cart, best_promo))

NameError: name 'fildelity' is not defined

### 7.4 변수 범위 규칙

In [14]:
def f1(a):
    print(a)
    print(b)
b=6
f1(3)

3
6


In [15]:
def f2(a):
    print(a)
    print(b)
    b =9
b=6
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [16]:
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [17]:
from dis import dis
print (dis(f1))
print (dis(f2))
print (dis(f3))

  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP
             20 LOAD_CONST               0 (None)
             23 RETURN_VALUE
None
  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  3          10 LOAD_GLOBAL              0 (print)
             13 LOAD_FAST                1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  4          20 LOAD_CONST               1 (9)
             23 STORE_FAST               1 (b)
             26 LOAD_CONST         

### 7.5 클로저
- 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수

In [27]:
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
avg = Averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


In [28]:
def make_averager():
    series = []
    def average(new_value):
        series.append(new_value)
        total=sum(series)
        return total/len(series)
    return average

avg = make_averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


In [29]:
print (avg.__code__.co_varnames)
print (avg.__code__.co_freevars)
print (avg.__closure__)
print (avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
(<cell at 0x7f3a55ab0b58: list object at 0x7f3a55246e48>,)
[10, 11, 12]


### 7.6 nonlocal 선언

In [1]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [2]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

avg = make_averager()
print (avg(10))
print (avg(11))
print (avg(12))

10.0
10.5
11.0


### 7.7 간단한 데커레이터 구현하기

In [4]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter()-t0
        name = func.__name__
        arg_str = ", ".join(repr(arg) for arg in args)
        print ("[%0.8fs] %s(%s) -> %r" %(elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)


print ("*"*40 , "Calling snooze(.123)")
snooze(.123)
print ("*"*40 , "Calling factorial(6)")
print ("6! = ", factorial(6))


**************************************** Calling snooze(.123)
[0.12372684s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000087s] factorial(1) -> 1
[0.00002763s] factorial(2) -> 2
[0.00004458s] factorial(3) -> 6
[0.00005983s] factorial(4) -> 24
[0.00007673s] factorial(5) -> 120
[0.00009316s] factorial(6) -> 720
6! =  720


### 7.7.1 작동 과정

In [11]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() -t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(", ".join(repr(arg) for arg in args))
        if kwargs:
            pairs = [ "%s=%r" %(k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(", ".join(pairs))
        arg_str = ", ".join(arg_lst)
        print ("[%0.8fs] %s(%s) -> %r " %(elapsed, name, arg_str, result))
        return result
    return clocked
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)


print ("*"*40 , "Calling snooze(.123)")
snooze(.123)
print ("*"*40 , "Calling factorial(6)")
print ("6! = ", factorial(6))


**************************************** Calling snooze(.123)
[0.12444043s] snooze(0.123) -> None 
**************************************** Calling factorial(6)
[0.00000143s] factorial(1) -> 1 
[0.00005102s] factorial(2) -> 2 
[0.00007200s] factorial(3) -> 6 
[0.00008917s] factorial(4) -> 24 
[0.00010800s] factorial(5) -> 120 
[0.00013161s] factorial(6) -> 720 
6! =  720


## 7.8 표준 라이브러리에서 제공하는 데커레이터
### 7.8.1 functools.lru_cache() 를 이용한 메모이제이션
- 메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다. 
- LRU (Least Recently Used) : 오랫동안 사용하지 않은 항목을 버림으로써 캐시가 무한정 커지지 않음을 의미한다.

In [15]:
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print (fibonacci(6))

[0.00000143s] fibonacci(0) -> 0 
[0.00000191s] fibonacci(1) -> 1 
[0.00027370s] fibonacci(2) -> 1 
[0.00000119s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000143s] fibonacci(1) -> 1 
[0.00005913s] fibonacci(2) -> 1 
[0.00011563s] fibonacci(3) -> 2 
[0.00044894s] fibonacci(4) -> 3 
[0.00000095s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000143s] fibonacci(1) -> 1 
[0.00005531s] fibonacci(2) -> 1 
[0.00011182s] fibonacci(3) -> 2 
[0.00000119s] fibonacci(0) -> 0 
[0.00000143s] fibonacci(1) -> 1 
[0.00005674s] fibonacci(2) -> 1 
[0.00000119s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000143s] fibonacci(1) -> 1 
[0.00005937s] fibonacci(2) -> 1 
[0.00011611s] fibonacci(3) -> 2 
[0.00022888s] fibonacci(4) -> 3 
[0.00039411s] fibonacci(5) -> 5 
[0.00090384s] fibonacci(6) -> 8 
8


In [16]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print (fibonacci(6))

[0.00000095s] fibonacci(0) -> 0 
[0.00000072s] fibonacci(1) -> 1 
[0.00011182s] fibonacci(2) -> 1 
[0.00000858s] fibonacci(3) -> 2 
[0.00016737s] fibonacci(4) -> 3 
[0.00000834s] fibonacci(5) -> 5 
[0.00022411s] fibonacci(6) -> 8 
8


### 7.8.2 단일 디스패치를 이용한 범용 함수

In [1]:
import html
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

print (htmlize({1,2,3}))
print (htmlize(abs))
print (htmlize("Heimlich & Co.\n- a game"))
print (htmlize(42))
print (htmlize(['alpha', 66, {3,2,1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co.\n- a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [2]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return "<pre>{}<pre>".format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace("\n", "<br>\n")
    return "<pre>{0}<pre>".format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return "<pre>{0} (0x{0:x})<pre>".format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = "</li>\n<li>".join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print (htmlize({1,2,3}))
print (htmlize(abs))
print (htmlize("Heimlich & Co.\n- a game"))
print (htmlize(42))
print (htmlize(['alpha', 66, {3,2,1}]))

<pre>{1, 2, 3}<pre>
<pre>&lt;built-in function abs&gt;<pre>
<pre>Heimlich &amp; Co.<br>
- a game<pre>
<pre>42 (0x2a)<pre>
<ul>
<li><pre>alpha<pre></li>
<li><pre>66 (0x42)<pre></li>
<li><pre>{1, 2, 3}<pre></li>
</ul>


## 7.9 누적된 데커레이터

## 7.10 매개변수화된 데커레이터

In [5]:
registry = []

def register(func):
    print ('running register(%s)' %func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print ('running main()')
print ('registry ->', registry)
f1()

running register(<function f1 at 0x7f28c1fc5f28>)
running main()
registry -> [<function f1 at 0x7f28c1fc5f28>]
running f1()


### 7.10.1 매개변수화된 등록 데커레이터

In [8]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' %(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print ("running f1()")
    
@register()
def f2():
    print ("running f2()")

def f3():
    print ("running f3()")

f1()
f2()
f3()

running register(active=False)->decorate(<function f1 at 0x7f28c1fe8620>)
running register(active=True)->decorate(<function f2 at 0x7f28d00a3158>)
running f1()
running f2()
running f3()


### 7.10.2 매개변수화된 clock 데커레이터

In [14]:
import time

DEFAULT_FMT ='[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time()-10
            name = func.__name__
            args=",".join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)
    
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

@clock('{name}({args})dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)


[1485354568.56486058s] snooze(0.123) -> None
[1485354568.68854499s] snooze(0.123) -> None
[1485354568.81207371s] snooze(0.123) -> None
snooze: 1485354568.9387553s
snooze: 1485354569.0629396s
snooze: 1485354569.1865883s
snooze(0.123)dt=1485354569.312s
snooze(0.123)dt=1485354569.436s
snooze(0.123)dt=1485354569.560s
